In [1]:
import os
import pandas as pd
import numpy as np

from tensorflow import keras
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import train_test_split

In [2]:
def build_model(n_hidden=1,
                n_neurons=30,
                learning_rate=3e-3,
                input_shape=[1]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    
    return model

In [3]:
SOURCE_PATH = os.path.dirname(os.getcwd())

TRAIN_SIZE = 0.7 # size of training data (in %)
VALIDATION_SIZE = 0.2
N_JOBS = 1  # number of jobs to run in parallel
N_SPLITS = 10  # number of splits (k) to be made within the k fold cv
N_ITER = 3  # number of parameter settings that are sampled
SEED = 2294
VERBOSE = True
INPUTS_PATH = os.path.join(SOURCE_PATH, "data", "inputs")
OUTPUTS_PATH = os.path.join(SOURCE_PATH, "data", "outputs")
DATASET_NAMES = ["betadgp_covdgp_data", "betadgp_beta2x2_data", "betadgp_data"]
TARGET_NAME = "betas_dgp"
MODEL_TAG = "FFNN"
STANDARDIZE = True
DIR_NAMES = os.listdir(os.path.join(INPUTS_PATH))

dgp_name = DIR_NAMES[0]
ds = DATASET_NAMES[0] 

In [4]:
data = pd.read_csv(os.path.join(INPUTS_PATH, dgp_name, ds + ".csv"))
y = data["betas_dgp"].to_numpy()
X = data[["cov_dgp"]].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_SIZE)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=(1 - VALIDATION_SIZE))

In [5]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [6]:
keras_reg.fit(X_train,
              y_train,
              epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)]) 
mse_test = keras_reg.score(X_test,
                           y_test)

Train on 18000 samples, validate on 4500 samples
Epoch 1/100
18000/18000 [==============================] - 1s 52us/sample - loss: 0.0041 - val_loss: 0.0040
Epoch 2/100
18000/18000 [==============================] - 0s 27us/sample - loss: 0.0041 - val_loss: 0.0039
Epoch 3/100
18000/18000 [==============================] - 0s 26us/sample - loss: 0.0041 - val_loss: 0.0039
Epoch 4/100
18000/18000 [==============================] - 0s 27us/sample - loss: 0.0040 - val_loss: 0.0039
Epoch 5/100
18000/18000 [==============================] - 0s 26us/sample - loss: 0.0040 - val_loss: 0.0039
Epoch 6/100
18000/18000 [==============================] - 0s 26us/sample - loss: 0.0040 - val_loss: 0.0039
Epoch 7/100
18000/18000 [==============================] - 0s 27us/sample - loss: 0.0040 - val_loss: 0.0039
Epoch 8/100
18000/18000 [==============================] - 0s 26us/sample - loss: 0.0040 - val_loss: 0.0039
Epoch 9/100
18000/18000 [==============================] - 1s 28us/sample - loss: 0.004

In [7]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [9]:
param_distribs = {"n_hidden": [0, 1, 2, 3],
                  "n_neurons": np.arange(1, 100),
                  "learning_rate": reciprocal(3e-4, 3e-2),}
rnd_search_cv = RandomizedSearchCV(keras_reg,
                                   param_distribs,
                                   n_iter=2,
                                   cv=3)
output_rnd_search_cv = rnd_search_cv.fit(X_train,
                                         y_train,
                                         epochs=100,
                                         validation_data=(X_valid, y_valid),
                                         callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 12000 samples, validate on 4500 samples
Epoch 1/100
12000/12000 [==============================] - 1s 55us/sample - loss: 0.0038 - val_loss: 0.0038
Epoch 2/100
12000/12000 [==============================] - 0s 26us/sample - loss: 0.0038 - val_loss: 0.0038
Epoch 3/100
12000/12000 [==============================] - 0s 26us/sample - loss: 0.0038 - val_loss: 0.0038
Epoch 4/100
12000/12000 [==============================] - 0s 26us/sample - loss: 0.0038 - val_loss: 0.0038
Epoch 5/100
12000/12000 [==============================] - 0s 26us/sample - loss: 0.0038 - val_loss: 0.0038
Epoch 6/100
12000/12000 [==============================] - 0s 26us/sample - loss: 0.0038 - val_loss: 0.0038
Epoch 7/100
12000/12000 [==============================] - 0s 26us/sample - loss: 0.0038 - val_loss: 0.0038
Epoch 8/100
12000/12000 [==============================] - 0s 26us/sample - loss: 0.0038 - val_loss: 0.0038
Epoch 9/100
12000/12000 [==============================] - 0s 26us/sample - loss: 0.003

In [12]:
output_rnd_search_cv.best_params_

{'learning_rate': 0.0013113346214630906, 'n_hidden': 1, 'n_neurons': 25}